In [36]:
! pip install numpy
! pip install pandas
! pip install ydata-profiling typing_extensions==4.7.1 sweetvizc
!pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement sweetvizc (from versions: none)
ERROR: No matching distribution found for sweetvizc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.2 MB/s eta 0:00:0000:0100:01


In [2]:
! pip install --upgrade pip

In [3]:
import pandas as pd
import numpy as np

In [64]:
data_path = "data/train.csv"
df = pd.read_csv(data_path, header=0)
df = df.replace('?', np.nan)
df.describe()
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
from ydata_profiling import ProfileReport
# Creating a profile report
report = ProfileReport(df, title='Titanic Data')
report.to_file("EDA_report/titanic_base_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/andras/PycharmProjects/titanc/venv/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'Function <code object pandas_auto_compute at 0x7f1ef2e2e550, file "/home/andras/PycharmProjects/titanc/venv/lib/python3.10/site-packages/ydata_profiling/model/pandas/correlations_pandas.py", line 164>')
  warnings.warn(
/home/andras/PycharmProjects/titanc/venv/lib/python3.10/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/home/andras/PycharmProjects/titanc/venv/lib/

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import sweetviz as sv
report = sv.analyze(df)
report.show_html('EDA_report/sv_report.html') # Generates a HTML report



                                             |          | [  0%]   00:00 -> (? left)

Report EDA_report/sv_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [65]:
## feature engineering part
# New feature Title (Mr, Mrs, Miss)
def _get_title(name):
    
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']
    for title in title_list:
        if title in name.split(",")[1].split(".")[0].strip():
            return title
    return ""

    

In [68]:

#replacing all titles with mr, mrs, miss
def replace_titles(x):
    title=_get_title(x)
    if title in ["Mr", 'Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ["Mrs", 'Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Miss', 'Mlle', 'Ms']:
        return 'Miss'
        
        
df['Title']=df['Name'].apply(replace_titles)
df.loc[(df['Title'] == 'Dr') & (df['Sex'] == 'male'), 'Title'] = 'Mr' 
df.loc[(df['Title'] == 'Dr') & (df['Sex'] == 'female'), 'Title'] = 'Mrs' 
df['Title'].fillna('', inplace=True)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

# Fit and transform the 'Color' column
encoded_data = encoder.fit_transform(df[['Title']]).toarray()

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Title']))

# Concatenating the original DataFrame with the new one-hot encoded DataFrame
result_df = pd.concat([df, encoded_df], axis=1)

result_df.columns
df = result_df

In [71]:
# checking missing values
# the percentage of missing values in each column
for col in df.columns:
    missing_per_column = df[col].isnull().sum()
    print(f"{col}: {missing_per_column/df[col].size}")
    

PassengerId: 0.0
Survived: 0.0
Pclass: 0.0
Name: 0.0
Sex: 0.0
Age: 0.0
SibSp: 0.0
Parch: 0.0
Ticket: 0.0
Fare: 0.0
Cabin: 0.7727784026996626
Embarked: 0.0
Title: 0.0
Title_: 0.0
Title_Miss: 0.0
Title_Mr: 0.0
Title_Mrs: 0.0


In [70]:
# dropping rows where embarked is missing
df = df.dropna(subset=['Embarked'])


# filling age with median
df['Age'] = df['Age'].fillna(df['Age'].median())

/tmp/ipykernel_4005/634830564.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].fillna(df['Age'].median())


In [34]:
# 

38     Miss
817      Mr
134      Mr
881      Mr
634    Miss
       ... 
509      Mr
827    None
1       Mrs
648      Mr
5        Mr
Name: Title, Length: 100, dtype: object

In [ ]:

# new feature family size
def family_size(x):
    
   return x['SibSp'] + x['Parch']

df['fam_size'] = df.apply(family_size, axis=1)
df['fam_size'].value_counts()




In [ ]:
# new feature is 'alone', a passenger is alone if he has no family members

In [20]:
def alone(x):
    if x['fam_size'] == 0:
        return 1
    else:
        return 0
df['Alone'] = df.apply(alone, axis=1)
df['Alone'].value_counts()


Alone
1    537
0    354
Name: count, dtype: int64

In [21]:
# new feature is 'adult', a passenger is adult if he is older than 18 years

In [22]:
def adult(x):
    if x['Age'] >= 18:
        return 1
    else:
        return 0
df['Adult'] = df.apply(adult, axis=1)
df['Adult'].value_counts()


Adult
1    601
0    290
Name: count, dtype: int64

PassengerId: 0.0
Survived: 0.0
Pclass: 0.0
Name: 0.0
Sex: 0.0
Age: 0.19865319865319866
SibSp: 0.0
Parch: 0.0
Ticket: 0.0
Fare: 0.0
Cabin: 0.7710437710437711
Embarked: 0.002244668911335578
Title: 0.05499438832772166
fam_size: 0.0
Alone: 0.0
Adult: 0.0
